In [ ]:
%load_ext autoreload
%autoreload 2

import os
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

DTYPE = torch.float16
model = qlib.load_custom_llama(
	model_name=model_name, 
	device_map='cpu', 
	torch_dtype=DTYPE, 
	#use_flash_attn=True
)

In [2]:
import qlib.qlayers
import qlib.qlayers.VQ2SQ


L=16
K=2
V=2
T=256
tlut_bits=10
decode_mode='LowBitSym' 
incoh_proc_mode='qtip' 

wrapper = qlib.HomeQuantWrapper(
    wrap_rule={
        torch.nn.Linear : qlib.qlayers.VQ2SQ.TrellisLinear(
			T=T, 
            L=L, 
            V=V, 
            K=K,
			tlut_bits=tlut_bits,
            decode_mode=decode_mode, 
            incoh_proc_mode=incoh_proc_mode,
			#viterby_bs=4096,
		),
    },
    exceptions = {
        'lm_head' : None,
    }
)
wrapper.wrap_model(model)

model_checkpoints_path = f'/home/msst/repo/Quantization/logs/checkpoints_{model_name}/trellis'
os.makedirs(model_checkpoints_path, exist_ok=True)
torch.save(model, os.path.join(model_checkpoints_path, f'T{T}_L{L}_V{V}_K{K}_lbits{tlut_bits}_{decode_mode}_{incoh_proc_mode}.pth'))


  0%|          | 0/16 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 GiB. GPU 0 has a total capacity of 11.64 GiB of which 9.28 GiB is free. Including non-PyTorch memory, this process has 2.34 GiB memory in use. Of the allocated memory 1.22 GiB is allocated by PyTorch, and 1.00 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)